In [2]:
import os
from PIL import Image

In [37]:
def create_augmented_image_paths(base_path, city_folder, x, y):
    base_folder = os.path.join(base_path, city_folder)
    augmented_folder = os.path.join(base_folder, "augment")
    if not os.path.exists(augmented_folder):
        os.makedirs(augmented_folder)
    #print(f"Augmented folder created: {augmented_folder}")
    return augmented_folder, [
        (x, y),
        (x + 1, y),
        (x, y + 1),
        (x + 1, y + 1)
    ]

def load_images(image_paths):
    images = []
    for path in image_paths:
        if os.path.exists(path):
            images.append(Image.open(path))
        else:
            print(f"Image not found: {path}")
            images.append(None)
    return images

def save_image(image, path):
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    image.save(path)
    #print(f"Image saved: {path}")

In [38]:


def augment_satellite_images(base_path, city_folders):
    for city_folder in city_folders:
        folder_path = os.path.join(base_path, city_folder)
        print(f"Processing city folder: {folder_path}")
        for file in os.listdir(folder_path):
            if file.endswith(".png"):
                base_name = file[:-4]
                parts = base_name.split('_')
                if len(parts) == 3:
                    x = int(parts[1])
                    y = int(parts[2])
                    augmented_folder, coords = create_augmented_image_paths(base_path, city_folder, x, y)
                    image_paths = [
                        os.path.join(folder_path, f"{parts[0]}_{coord[0]}_{coord[1]}.png")
                        for coord in coords
                    ]
                    images = load_images(image_paths)
                    #print(images)

                    # 生成右平移图像
                    if images[0] is not None and images[1] is not None:
                        right_shift = Image.new('RGB', (images[0].width, images[0].height))
                        right_shift.paste(images[0].crop((images[0].width // 2, 0, images[0].width, images[0].height)), (0, 0))
                        right_shift.paste(images[1].crop((0, 0, images[0].width // 2, images[0].height)), (images[0].width // 2, 0))
                        right_shift_name = f"{parts[0]}_{x}+5_{y}+0.png"
                        save_image(right_shift, os.path.join(augmented_folder, right_shift_name))

                    # 生成下平移图像
                    if images[0] is not None and images[2] is not None:
                        down_shift = Image.new('RGB', (images[0].width, images[0].height))
                        down_shift.paste(images[0].crop((0, images[0].height // 2, images[0].width, images[0].height)), (0, 0))
                        down_shift.paste(images[2].crop((0, 0, images[0].width, images[0].height // 2)), (0, images[0].height // 2))
                        down_shift_name = f"{parts[0]}_{x}+0_{y}+5.png"
                        save_image(down_shift, os.path.join(augmented_folder, down_shift_name))

                    # 生成右下平移图像
                    if images[0] is not None and images[1] is not None and images[2] is not None and images[3] is not None:
                        #print(f"Generating right down shift image for base name: {base_name}")
                        right_down_shift = Image.new('RGB', (images[0].width, images[0].height))
                        right_down_shift.paste(images[0].crop((images[0].width // 2, images[0].height // 2, images[0].width, images[0].height)), (0, 0))
                        right_down_shift.paste(images[1].crop((0, images[0].height // 2, images[0].width // 2, images[0].height)), (images[0].width // 2, 0))
                        right_down_shift.paste(images[2].crop((images[0].width // 2, 0, images[0].width, images[0].height // 2)), (0, images[0].height // 2))
                        right_down_shift.paste(images[3].crop((0, 0, images[0].width // 2, images[0].height // 2)), (images[0].width // 2, images[0].height // 2))
                        right_down_shift_name = f"{parts[0]}_{x}+5_{y}+5.png"
                        save_image(right_down_shift, os.path.join(augmented_folder, right_down_shift_name))
                    else:
                        print(f"Not enough images found for right down shift for base name: {base_name}")

def augment_constraint_images(base_path, city_folders):
    for city_folder in city_folders:
        folder_path = os.path.join(base_path, city_folder)
        print(f"Processing constraint city folder: {folder_path}")
        for x_folder in os.listdir(folder_path):
            x_path = os.path.join(folder_path, x_folder)
            if os.path.isdir(x_path):
                x = int(x_folder)
                augmented_x_folder_0 = os.path.join(folder_path, f"{x}+0")
                augmented_x_folder_1 = os.path.join(folder_path, f"{x}+5")
                if not os.path.exists(augmented_x_folder_0):
                    os.makedirs(augmented_x_folder_0)
                    print(f"Augmented x+0 folder created: {augmented_x_folder_0}")
                if not os.path.exists(augmented_x_folder_1):
                    os.makedirs(augmented_x_folder_1)
                    print(f"Augmented x+1 folder created: {augmented_x_folder_1}")
                for y_file in os.listdir(x_path):
                    if y_file.endswith(".png"):
                        y = int(y_file[:-4])
                        base_image = Image.open(os.path.join(x_path, y_file))

                        # 生成右平移图像
                        if base_image:
                            right_shift = Image.new('RGB', (base_image.width, base_image.height))
                            right_shift.paste(base_image.crop((base_image.width // 2, 0, base_image.width, base_image.height)), (0, 0))
                            right_shift.paste(base_image.crop((0, 0, base_image.width // 2, base_image.height)), (base_image.width // 2, 0))
                            right_shift_name = f"{y}+0.png"
                            save_image(right_shift, os.path.join(augmented_x_folder_1, right_shift_name))

                        # 生成下平移图像
                        if base_image:
                            down_shift = Image.new('RGB', (base_image.width, base_image.height))
                            down_shift.paste(base_image.crop((0, base_image.height // 2, base_image.width, base_image.height)), (0, 0))
                            down_shift.paste(base_image.crop((0, 0, base_image.width, base_image.height // 2)), (0, base_image.height // 2))
                            down_shift_name = f"{y}+5.png"
                            save_image(down_shift, os.path.join(augmented_x_folder_0, down_shift_name))

                        # 生成右下平移图像
                        if base_image:
                            right_down_shift = Image.new('RGB', (base_image.width, base_image.height))
                            right_down_shift.paste(base_image.crop((base_image.width // 2, base_image.height // 2, base_image.width, base_image.height)), (0, 0))
                            right_down_shift.paste(base_image.crop((0, base_image.height // 2, base_image.width // 2, base_image.height)), (base_image.width // 2, 0))
                            right_down_shift.paste(base_image.crop((base_image.width // 2, 0, base_image.width, base_image.height // 2)), (0, base_image.height // 2))
                            right_down_shift.paste(base_image.crop((0, 0, base_image.width // 2, base_image.height // 2)), (base_image.width // 2, base_image.height // 2))
                            right_down_shift_name = f"{y}+5.png"
                            save_image(right_down_shift, os.path.join(augmented_x_folder_1, right_down_shift_name))



In [ ]:
# 设置基础路径和城市文件夹名称
base_path = '/Users/hemingyi/Research/MIT-UF-NEU_2024/GenerativeUrbanDesign'
city_folders_satellite = ['chicago_satellite', 'dallas_satellite', 'la_satellite']
city_folders_constraint = ['chicago_constraint', 'dallas_constraint', 'la_constraint']



In [8]:
# 设置基础路径和城市文件夹名称
base_path = '/Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images'
city_folders_satellite = ['chicago_satellite']
#city_folders_constraint = ['chicago_constraint', 'dallas_constraint', 'la_constraint']



In [39]:
# 处理卫星图像
augment_satellite_images(base_path, city_folders_satellite)

Processing city folder: /Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images/chicago_satellite
Image not found: /Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images/chicago_satellite/16_16644_24498.png
Image not found: /Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images/chicago_satellite/16_16645_24498.png
Not enough images found for right down shift for base name: 16_16644_24497
Image not found: /Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images/chicago_satellite/16_16648_24306.png
Image not found: /Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images/chicago_satellite/16_16648_24307.png
Not enough images found for right down shift for base name: 16_16647_24306
Image not found: /Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images/chicago_satellite/16_16648_24312.png
Image not found: /Users/hemingyi/Research/MIT-UF-NEU_2024/Try/Satellite_Images/chicago_satellite/16_16648_24313.png
Not enough images found for right down shift for b

In [ ]:
# 处理约束图像
augment_constraint_images(base_path, city_folders_constraint)